# Development Notebook for Cortical Crowding Project

In [ ]:
import os, sys
from pathlib import Path

import numpy as np
import pandas as pd
import neuropythy as ny
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.optimize
from scipy.stats import gmean
from scipy.optimize import curve_fit

In [ ]:
# We want to be able to load in libraries that are in this repository's src directory,
# so we add src to the system path:
sys.path.append('../src')

# Now we can import corticalcrowding from the src directory:
import corticalcrowding as cc

In [ ]:
df = pd.read_csv('/data/crowding/crowding_data_withID.csv')

In [ ]:
sids_NYU = [
    'sub-wlsubj070',
    'sub-wlsubj114',
    'sub-wlsubj121',
    'sub-wlsubj135']
sids_NEI = [ 
    'sub-wlsubj119',
    'sub-wlsubj127',
    'sub-wlsubj136',
    'sub-wlsubj137',
    'sub-wlsubj143',
    'sub-wlsubj144',
    'sub-wlsubj145',
    'sub-wlsubj146',
    'sub-wlsubj147',
    'sub-wlsubj148',
    'sub-wlsubj149',
    'sub-wlsubj150',
    'sub-wlsubj151',
    'sub-wlsubj152',
    'sub-wlsubj153',
    'sub-wlsubj154',
    'sub-wlsubj155',
    'sub-wlsubj156',
    'sub-wlsubj157',
    'sub-wlsubj158',
    'sub-wlsubj159',
    'sub-wlsubj160',
    'sub-wlsubj161',
    'sub-wlsubj162',
    'sub-wlsubj163',
    'sub-wlsubj164',
    'sub-wlsubj165',
    'sub-wlsubj166',
    'sub-wlsubj167',
    'sub-wlsubj168',
    'sub-wlsubj170',
    'sub-wlsubj171',
    'sub-wlsubj172',
    'sub-wlsubj173',
    'sub-wlsubj174',
    'sub-wlsubj175',
    'sub-wlsubj176']

In [ ]:
sids_orig = sids_NYU + sids_NEI

## crowding

In [ ]:
# merge the 2 sessions, so now each sub have 12 cd (match with polar angle) instead of 24
mean_cd_polar = df.groupby(['ID','Eccen_X','Eccen_Y'])['CrowdingDistance'].apply(gmean).reset_index()

In [ ]:
# each subject has 1 cd value at each eccentricity
mean_cd = df.groupby(['ID','RadialEccentricity'])['CrowdingDistance'].apply(gmean).reset_index()

In [ ]:
cd_list = df['CrowdingDistance'].tolist()
mean_cd_list=mean_cd['CrowdingDistance'].tolist()

In [ ]:
# create 3 dfs based on eccen
mean_1 = mean_cd[mean_cd['RadialEccentricity']==2.5]
m_1 = mean_1['CrowdingDistance'].mean()
st_1 = mean_1['CrowdingDistance'].std()
mean_2 = mean_cd[mean_cd['RadialEccentricity']==5]
m_2 = mean_2['CrowdingDistance'].mean()
st_2 = mean_2['CrowdingDistance'].std()
mean_3 = mean_cd[mean_cd['RadialEccentricity']==10]
m_3 = mean_3['CrowdingDistance'].mean()
st_3 = mean_3['CrowdingDistance'].std()

In [ ]:
x_ecc = df['RadialEccentricity'].tolist()
mean_x_ecc = mean_cd['RadialEccentricity'].tolist()

In [ ]:
b, _ = curve_fit(cc.crowding.func_cd, x_ecc, np.log10(cd_list), p0=0.15)
b

In [ ]:
mean_values = [m_1, m_2, m_3]
std_values = [st_1, st_2, st_3]
eccentricities = [2.5, 5, 10]

In [ ]:
mean_values

## calculate cortical magnification

In [ ]:
eccen = np.linspace(1, 11, 1000)

In [ ]:
all_cmag_v1 = []
all_cmag_v2 = []
all_cmag_v3 = []
all_cmag_v4 = []
all_eccen_v1 = []
all_eccen_v2 = []
all_eccen_v3 = []
all_eccen_v4 = []
all_mask = ('variance_explained', 0.04, 1)

for sid in sids_orig:
    try:
        sub = cc.cmag.load_subject(sid)
        print(sid)
        # Calculate cmag for the subject for V1
        v1_mask = {'and': [('visual_area', 1), all_mask]}
        eccen_v1, cmag_v1 = cc.cmag.ring_cmag(sub, eccen=None, mask=v1_mask)
        all_eccen_v1.append(eccen_v1)
        all_cmag_v1.append(cmag_v1)

        # Calculate cmag for the subject for V2
        v2_mask = {'and': [('visual_area', 2), all_mask]}
        eccen_v2, cmag_v2 = cc.cmag.ring_cmag(sub, eccen=None, mask=v2_mask)
        all_eccen_v2.append(eccen_v2)
        all_cmag_v2.append(cmag_v2)

        # Calculate cmag for the subject for V3
        v3_mask = {'and': [('visual_area', 3), all_mask]}
        eccen_v3, cmag_v3 = cc.cmag.ring_cmag(sub, eccen=None, mask=v3_mask)
        all_eccen_v3.append(eccen_v3)
        all_cmag_v3.append(cmag_v3)
        
        # Calculate cmag for the subject for V4
        v4_mask = {'and': [('visual_area', 4), all_mask]}
        eccen_v4, cmag_v4 = cc.cmag.ring_cmag(sub, eccen=None, mask=v4_mask)
        all_eccen_v4.append(eccen_v4)
        all_cmag_v4.append(cmag_v4)
        
    
    except Exception as e:
        print(f"Error calculating cmag for subject {sid}: {e}")

all_flatcmag_v1 = np.concatenate(all_cmag_v1)
all_flateccen_v1 = np.concatenate(all_eccen_v1)
all_flatcmag_v2 = np.concatenate(all_cmag_v2)
all_flateccen_v2 = np.concatenate(all_eccen_v2)
all_flatcmag_v3 = np.concatenate(all_cmag_v3)
all_flateccen_v3 = np.concatenate(all_eccen_v3)
all_flatcmag_v4 = np.concatenate(all_cmag_v4)
all_flateccen_v4 = np.concatenate(all_eccen_v4)

In [ ]:
ii = (all_flateccen_v1 < 11) & (all_flatcmag_v1 > 0) & (all_flateccen_v1 > 0.2)
popt1 = cc.cmag.fit_cmag(all_flateccen_v1[ii], all_flatcmag_v1[ii],method='BFGS')
popt1_3params = cc.cmag.fit_cmag(all_flateccen_v1[ii], all_flatcmag_v1[ii],p0=[17.3,0.75,2],method='BFGS')
print(popt1,popt1_3params)
plt.loglog(all_flateccen_v1,all_flatcmag_v1,'k.',alpha=0.01) 
plot_ecc = np.sort(all_flateccen_v1)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt1),'r-', zorder=10)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt1_3params),'c-', zorder=10)
plt.show()

In [ ]:
ii = (all_flateccen_v2 < 11) & (all_flatcmag_v2 > 0) & (all_flateccen_v2 > 0.2)
popt2 = cc.cmag.fit_cmag(all_flateccen_v2[ii], all_flatcmag_v2[ii],method='BFGS')
popt2_3params = cc.cmag.fit_cmag(all_flateccen_v2[ii], all_flatcmag_v2[ii],p0=[17.3,0.75,2],method='BFGS')
print(popt2,popt2_3params)
plt.loglog(all_flateccen_v2,all_flatcmag_v2,'k.',alpha=0.01) 
plot_ecc = np.sort(all_flateccen_v2)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt2),'r-', zorder=10)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt2_3params),'c-', zorder=10)
plt.show()

In [ ]:
ii = (all_flateccen_v3 < 11) & (all_flatcmag_v3 > 0) & (all_flateccen_v3 > 0.2)
popt3 = cc.cmag.fit_cmag(all_flateccen_v3, all_flatcmag_v3,method='BFGS')
popt3_3params = cc.cmag.fit_cmag(all_flateccen_v3[ii], all_flatcmag_v3[ii],p0=[17.3,0.75,2],method='BFGS')
print(popt3,popt3_3params)
plt.loglog(all_flateccen_v3,all_flatcmag_v3,'k.',alpha=0.01) 
plot_ecc = np.sort(all_flateccen_v3)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt3),'r-', zorder=10)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt3_3params),'c-', zorder=10)
plt.show()

In [ ]:
ii = (all_flateccen_v4 < 11) & (all_flatcmag_v4 > 0) & (all_flateccen_v4 > 0.2)
popt4 = cc.cmag.fit_cmag(all_flateccen_v4[ii], all_flatcmag_v4[ii],method='BFGS')
popt4_3params = cc.cmag.fit_cmag(all_flateccen_v4[ii], all_flatcmag_v4[ii],p0=[17.3,0.75,2],method='BFGS')
print(popt4,popt4_3params)
plt.loglog(all_flateccen_v4,all_flatcmag_v4,'k.',alpha=0.01) 
plot_ecc = np.sort(all_flateccen_v4)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt4),'r-', zorder=10)
plt.loglog(plot_ecc, cc.cmag.HH91(plot_ecc, *popt4_3params),'c-', zorder=10)
plt.show()

In [ ]:
eccen = np.linspace(0.5, 11, 1000)
plt.loglog(eccen, cc.cmag.HH91(eccen),'k-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt1),'b-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt2),'r-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt3),'c-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt4),'g-')
plt.show()

In [ ]:
eccen = np.linspace(0.5, 11, 1000)
plt.loglog(eccen, cc.cmag.HH91(eccen),'k-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt1_3params),'b-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt2_3params),'r-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt3_3params),'c-')
plt.loglog(eccen, cc.cmag.HH91(eccen, *popt4_3params),'g-')
plt.show()

## add cortical magnification values to df

In [ ]:
# magnification_values is a dict containing cmag for each visual area
magnification_values = cc.cmag.calculate_cortical_magnification(df)

for mask_value in [1, 2, 3, 4]:
    col_name = f'CorticalMagnification_{mask_value}'
    df.loc[:, col_name] = magnification_values[mask_value]

In [ ]:
df_sub_polar = df.groupby(['Eccen_X','Eccen_Y','ID'])[[
    'CorticalMagnification_1',
    'CorticalMagnification_2',
    'CorticalMagnification_3',
    'CorticalMagnification_4'
]].mean().reset_index()

In [ ]:
df_cell = df_sub_polar
# convert 2d cortical magnification into 1d visual magnification and change the names of columns accordingly
for col in ['CorticalMagnification_1', 'CorticalMagnification_2', 'CorticalMagnification_3', 'CorticalMagnification_4']:
    df_cell[col] = np.sqrt(1 / df_cell[col])
df_cell.rename(columns={
    'CorticalMagnification_1': 'VisualMagnification_1',
    'CorticalMagnification_2': 'VisualMagnification_2',
    'CorticalMagnification_3': 'VisualMagnification_3',
    'CorticalMagnification_4': 'VisualMagnification_4'
}, inplace=True)

In [ ]:
# sort the df
df_cell.sort_values(by=['ID', 'Eccen_X','Eccen_Y'], inplace=True)
# add the crowding values to the df
mean_for_sub = mean_cd_polar['CrowdingDistance'].values
df_cell['CrowdingDistance'] = mean_for_sub

In [ ]:
# drop nan values
df_sub_clean = df_cell.dropna(subset=['VisualMagnification_1'])

In [ ]:
mean_cd_polar[mean_cd_polar['ID']=='sub-wlsubj135']

In [ ]:
df_sub_clean[df_sub_clean['ID']=='sub-wlsubj135']

## plot visual magnification and crowding values 

In [ ]:
sids = np.unique(df_sub_clean['ID'])
fig, axes = plt.subplots((len(sids)+1)//2,2, figsize=(8,16))
x_ticks = [-10, -5, -2.5, 2.5, 5, 10]
for sid, ax in zip(sids, axes.flat):
    subject_data = df_sub_clean[(df_sub_clean['ID']==sid) & (df_sub_clean['Eccen_Y']==0)]
    y1 = subject_data['VisualMagnification_1'].values
    y2 = subject_data['VisualMagnification_2'].values
    y3 = subject_data['VisualMagnification_3'].values
    y4 = subject_data['VisualMagnification_4'].values
    y5 = subject_data['CrowdingDistance'].values
    
    ax.plot(subject_data['Eccen_X'], y1, 'bo-')
    ax.plot(subject_data['Eccen_X'], y2, 'ro-')
    ax.plot(subject_data['Eccen_X'], y3, 'co-')
    ax.plot(subject_data['Eccen_X'], y4, 'go-')
    ax.plot(subject_data['Eccen_X'], y5, 'ko-')
    
    ax.set_xticks(x_ticks)
    ax.set_title(sid)
    ax.set_xlabel("Horizontal meridian")
    
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

In [ ]:
fig, axes = plt.subplots((len(sids)+1)//2,2, figsize=(8,16))
x_ticks = [-10, -5, -2.5, 2.5, 5, 10]

for sid, ax in zip(sids, axes.flat):
    subject_data = df_sub_clean[(df_sub_clean['ID']==sid) & (df_sub_clean['Eccen_X']==0)]
    y1 = subject_data['VisualMagnification_1'].values
    y2 = subject_data['VisualMagnification_2'].values
    y3 = subject_data['VisualMagnification_3'].values
    y4 = subject_data['VisualMagnification_4'].values
    y5 = subject_data['CrowdingDistance'].values
    
    ax.plot(subject_data['Eccen_Y'], y1, 'bo-')
    ax.plot(subject_data['Eccen_Y'], y2, 'ro-')
    ax.plot(subject_data['Eccen_Y'], y3, 'co-')
    ax.plot(subject_data['Eccen_Y'], y4, 'go-')
    ax.plot(subject_data['Eccen_Y'], y5, 'ko-')
    
    ax.set_xticks(x_ticks)
    ax.set_title(sid)
    ax.set_xlabel("Vertical meridian")
    
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

## use cortical magnification to predict crowding

In [ ]:
sids = np.unique(df_sub_clean['ID'])

fig, axes = plt.subplots((len(sids)+1)//2,2, figsize=(8,16))

for sid, ax in zip(sids, axes.flat):
    subject_data = df_sub_clean[df_sub_clean['ID']==sid]
    y1 = subject_data['VisualMagnification_1'].values
    y2 = subject_data['VisualMagnification_2'].values
    y3 = subject_data['VisualMagnification_3'].values
    y4 = subject_data['VisualMagnification_4'].values
    y5 = subject_data['CrowdingDistance'].values

    rss_y1, coef_y1 = cc.regression.fit_and_evaluate(y1, y5)
    rss_y2, coef_y2 = cc.regression.fit_and_evaluate(y2, y5)
    rss_y3, coef_y3 = cc.regression.fit_and_evaluate(y3, y5)
    rss_y4, coef_y4 = cc.regression.fit_and_evaluate(y4, y5)

    ax.plot(y1, y5, 'ro-')
    ax.plot(y2, y5, 'go-')
    ax.plot(y3, y5, 'bo-')
    ax.plot(y4, y5, 'ko-')
    ax.scatter([0],[0],color = '0.5')
#ax.axis('equal')

In [ ]:
grouped = df_sub_clean.groupby('ID')
subject_results = {}

# Iterate over each subject to do the regression
for subject, subject_data in grouped:
    y1 = subject_data['VisualMagnification_1'].values
    y2 = subject_data['VisualMagnification_2'].values
    y3 = subject_data['VisualMagnification_3'].values
    y4 = subject_data['VisualMagnification_4'].values
    y5 = subject_data['CrowdingDistance'].values

    
    rss_y1, coef_y1 = cc.regression.fit_and_evaluate(y1, y5)
    rss_y2, coef_y2 = cc.regression.fit_and_evaluate(y2, y5)
    rss_y3, coef_y3 = cc.regression.fit_and_evaluate(y3, y5)
    rss_y4, coef_y4 = cc.regression.fit_and_evaluate(y4, y5)
    
    rss_values = [rss_y1, rss_y2, rss_y3, rss_y4]
    best_fit_index = np.argmin(rss_values)
    best_fit = ['y1','y2','y3','y4'][best_fit_index]
    
    # create a dict to store values
    subject_results[subject] = {
        'rss_y1': rss_y1, 'coef_y1': coef_y1,
        'rss_y2': rss_y2, 'coef_y2': coef_y2,
        'rss_y3': rss_y3, 'coef_y3': coef_y3,
        'rss_y4': rss_y4, 'coef_y4': coef_y4,
        'best_fit': best_fit
    }

for subject, results in subject_results.items():
    print(f"Results for {subject}:")
    print(f"RSS for y1: {results['rss_y1']}, Coefficient: {results['coef_y1']}")
    print(f"RSS for y2: {results['rss_y2']}, Coefficient: {results['coef_y2']}")
    print(f"RSS for y3: {results['rss_y3']}, Coefficient: {results['coef_y3']}")
    print(f"RSS for y4: {results['rss_y4']}, Coefficient: {results['coef_y4']}")
    print(f"The best fit is: {results['best_fit']}\n")


In [ ]:
key = 'rss_y1'
data = [[subdata[key] for subdata in subject_results.values()]
        for key in ['rss_y1','rss_y2','rss_y3','rss_y4']]
plt.hist(data)
plt.show()

In [ ]:
rss_y1_values = []
rss_y2_values = []
rss_y3_values = []
rss_y4_values = []

for subject, values in subject_results.items():
    rss_y1_values.append(values['rss_y1'])
    rss_y2_values.append(values['rss_y2'])
    rss_y3_values.append(values['rss_y3'])
    rss_y4_values.append(values['rss_y4'])

mean_rss_y1 = np.mean(rss_y1_values)
mean_rss_y2 = np.mean(rss_y2_values)
mean_rss_y3 = np.mean(rss_y3_values)
mean_rss_y4 = np.mean(rss_y4_values)

std_rss_y1 = np.std(rss_y1_values)
std_rss_y2 = np.std(rss_y2_values)
std_rss_y3 = np.std(rss_y3_values)
std_rss_y4 = np.std(rss_y4_values)

# Print results
print(f"Mean RSS for y1: {mean_rss_y1}, Standard Deviation: {std_rss_y1}")
print(f"Mean RSS for y2: {mean_rss_y2}, Standard Deviation: {std_rss_y2}")
print(f"Mean RSS for y3: {mean_rss_y3}, Standard Deviation: {std_rss_y3}")
print(f"Mean RSS for y4: {mean_rss_y4}, Standard Deviation: {std_rss_y4}")

In [ ]:
mean_rss = [mean_rss_y1, mean_rss_y2, mean_rss_y3, mean_rss_y4]
std_rss = [std_rss_y1, std_rss_y2, std_rss_y3, std_rss_y4]

labels = ['V1', 'V2', 'V3', 'V4']

plt.figure(figsize=(8, 6))
plt.bar(labels, mean_rss, yerr=std_rss/np.sqrt(len(sids)))

plt.ylabel('RSS (Residual Sum of Squares)')
plt.title('Mean RSS with Standard Error Across Visual Areas')
plt.show()

In [ ]:
## look at group data

In [ ]:
grouped_df = df.groupby('RadialEccentricity')[[
    'CorticalMagnification_1',
    'CorticalMagnification_2',
    'CorticalMagnification_3',
    'CorticalMagnification_4'
]].mean().reset_index()

# make it 1d visual magnification
for col in ['CorticalMagnification_1', 'CorticalMagnification_2', 'CorticalMagnification_3', 'CorticalMagnification_4']:
    grouped_df[col] = np.sqrt(1 / grouped_df[col])

In [ ]:
grouped_df['CrowdingDistance'] = mean_values
grouped_df.rename(columns={
    'CorticalMagnification_1': 'VisualMagnification_1',
    'CorticalMagnification_2': 'VisualMagnification_2',
    'CorticalMagnification_3': 'VisualMagnification_3',
    'CorticalMagnification_4': 'VisualMagnification_4'
}, inplace=True)
grouped_df

In [ ]:
plt.figure(figsize=(10, 6))
for col in ['VisualMagnification_1', 'VisualMagnification_2', 'VisualMagnification_3', 'VisualMagnification_4']:
    plt.scatter(grouped_df['RadialEccentricity'], grouped_df[col], marker='o', linestyle='-', label=col)
plt.scatter(eccentricities, mean_values, color='cyan', label = 'crowding distance')
plt.xlabel('Radial Eccentricity')
plt.ylabel('Visual Magnification')
plt.title('Visual Magnification vs Radial Eccentricity')
plt.legend()
plt.show()

In [ ]:
y1 = grouped_df['VisualMagnification_1'].values
y2 = grouped_df['VisualMagnification_2'].values
y3 = grouped_df['VisualMagnification_3'].values
y4 = grouped_df['VisualMagnification_4'].values
y5 = grouped_df['CrowdingDistance'].values

rss_y1, coef_y1 = cc.regression.fit_and_evaluate(y1, y5)
rss_y2, coef_y2 = cc.regression.fit_and_evaluate(y2, y5)
rss_y3, coef_y3 = cc.regression.fit_and_evaluate(y3, y5)
rss_y4, coef_y4 = cc.regression.fit_and_evaluate(y4, y5)

rss_values = [rss_y1, rss_y2, rss_y3, rss_y4]
best_fit_index = np.argmin(rss_values)
best_fit = ['y1', 'y2', 'y3', 'y4'][best_fit_index]

print(f"RSS for y1: {rss_y1}, Coefficient: {coef_y1}")
print(f"RSS for y2: {rss_y2}, Coefficient: {coef_y2}")
print(f"RSS for y3: {rss_y3}, Coefficient: {coef_y3}")
print(f"RSS for y4: {rss_y4}, Coefficient: {coef_y4}")
print(f"The best fit is: {best_fit}")

In [ ]:
plt.figure(figsize=(10, 6))
for scale, col in zip([coef_y1,coef_y2,coef_y3,coef_y4],['VisualMagnification_1', 'VisualMagnification_2', 'VisualMagnification_3', 'VisualMagnification_4']):
    plt.scatter(grouped_df['RadialEccentricity'], scale * grouped_df[col], marker='o', linestyle='-', label=col)
plt.scatter(eccentricities, mean_values, color='cyan', label = 'crowding distance')
plt.xlabel('Radial Eccentricity')
plt.ylabel('Scaled Visual Magnification')
plt.title('Scaled Visual Magnification vs Radial Eccentricity')
plt.legend()
plt.show()

## Testing alternative fitting

In [ ]:
def HH91_vmag(ecc, a=17.3, b=0.75, c=2):
    return ((np.asarray(ecc) + b) / a)**c

def fit_vmag(ecc, cmag, params0=(17.3, 0.75), method=None):
    from scipy.optimize import minimize
    ecc = np.asarray(ecc, dtype=np.float64)
    cmag = np.asarray(cmag, dtype=np.float64)
    vmag = 1/cmag
    def loss_vmag(params):
        params = list(params)
        params[1] = params[1]**2
        pred = HH91_vmag(ecc, *params)
        error = (pred - vmag)
        return np.sum(error**2)
    params0 = list(params0)
    params0[1] = np.sqrt(params0[1])
    r = minimize(loss_vmag, params0, method=method)
    r.x[1] = r.x[1]**2
    r.coords = np.array([ecc,cmag])
    return r

def trimsame(x, *args):
    x = np.asarray(x)
    idx0 = np.where(x != x[0])[0][0] - 1
    idxn = np.where(~np.isclose(x, x[-1]))[0][-1] + 1
    return (x[idx0:idxn],) + tuple(arg[idx0:idxn] for arg in args)

def fit_subject_vmag(sid,
                     params0=(17.3, 0.75),
                     method=None,
                     allmask={'and': [('variance_explained', 0.04, 1),
                                      ('eccentricity', 0, 12)]}):
    sub = cc.cmag.load_subject(sid)
    cmags = []
    eccens = []
    results = []
    for k in range(1,5):
        mask = {'and': [('visual_area', k), allmask]}
        eccen, cmag = cc.cmag.ring_cmag(sub, mask=mask)
        (cmag, eccen) = trimsame(cmag, eccen)
        eccens.append(eccen)
        cmags.append(cmag)
        results.append(fit_vmag(eccen, cmag, params0, method=method))
    return results

In [ ]:
rs = fit_subject_vmag('sub-wlsubj150')

In [ ]:
(fig,ax) = plt.subplots(1,1, figsize=(5,3), dpi=288)

eccrng = np.linspace(0, 11, 500)
for (clr,r) in zip(['r','g','b','k'], rs):
    #if clr != 'r': continue
    (ecc,cmag) = r.coords
    if len(r.x) == 3:
        (a,b,c) = r.x
    else:
        (a,b) = r.x
        c = 2
    ax.loglog(ecc, cmag, clr + '.')
    ax.loglog(eccrng, 1/HH91_vmag(eccrng, a, b, c), clr + '-')
ax.set_ylim([0.5,512])

In [ ]:
def cmag_basics(sid, h, label):
    sub = cc.cmag.load_subject(sid)
    hem = sub.hemis[h]
    mask_nor2 = {
        'and': [
            ('eccentricity', 0, 12),
            ('visual_area', label)]}
    mask_r2 = {'and': mask_nor2['and'] + [('variance_explained', 0.04, 1)]}
    rdat = ny.retinotopy_data(hem)
    mask_ii = hem.mask(mask_r2)
    ecc = rdat['eccentricity'][mask_ii]
    srf = hem.prop('midgray_surface_area')
    totarea = np.sum(srf[hem.mask(mask_nor2)])
    srf = srf[mask_ii]
    ii = np.argsort(ecc)
    ecc = ecc[ii]
    srf = srf[ii] * totarea / np.sum(srf)
    return (ecc, srf)

$$ c_1 = \sqrt{\frac{a_M}{\pi \left(\log\left(\frac{c_2 + M}{c_2}\right) - \frac{M}{c_2 + M}\right)}} $$

In [ ]:
def HH91_integral(x, a=17.3, b=0.75):
    xb = x + b
    return a**2 * np.pi * (np.log(xb / b) - x / xb)

def HH91_c1(totalarea, maxecc, b=0.75):
    mb = maxecc + b
    c1 = np.sqrt(totalarea / np.pi / (np.log(mb / b) - maxecc/mb))
    
def fit_cumarea_data(ecc, srf, params0=(17.3, 0.75), method=None):
    from scipy.optimize import minimize
    ecc = np.asarray(ecc, dtype=np.float64)
    srf = np.asarray(srf, dtype=np.float64)
    ii = np.argsort(ecc)
    ecc = ecc[ii]
    srf = srf[ii]
    cumsrf = np.cumsum(srf)
    def loss_vmag(params):
        params = list(params)
        params[1] = params[1]**2
        pred = HH91_integral(ecc, *params)
        error = (pred - cumsrf)
        return np.mean(error**2)
    params0 = list(params0)
    params0[1] = np.sqrt(params0[1])
    r = minimize(loss_vmag, params0, method=method)
    r.x[0] = abs(r.x[0])
    r.x[1] = r.x[1]**2
    r.coords = np.array([ecc,srf])
    return r

def fit_cumarea(sid, h, label):
    (ecc,srf) = cmag_basics(sid, h, label)
    if len(ecc) == 0:
        raise RuntimeError(f"no data found for {sid}:{h}:{label}")
    r = fit_cumarea_data(ecc, srf)
    r.coords = np.array([ecc, srf])
    return r

In [ ]:
sid = 'sub-wlsubj136'

(fig,axs) = plt.subplots(1,2, figsize=(7,3), dpi=288)
eccrng = np.linspace(0, 11, 500)

for (h,ax) in zip(['lh','rh'], axs):
    for (clr, lbl) in zip(['r','g','b','k'], [1,2,3,4]):
        if lbl != 4:
            continue
        r = fit_cumarea(sid, h, lbl)
        (ecc,srf) = r.coords
        (a,b) = r.x
        print(f"{sid:<13s} {h:2s} V{lbl}: a={a:5.2f}, b={b:5.3f}")
        ax.plot(ecc, np.cumsum(srf), clr + '.', lw=0.5, alpha=0.05)
        ax.plot(ecc, HH91_integral(ecc, a, b), clr + '-', lw=0.5)

In [ ]:
df = dict(sid=[], h=[], label=[], a=[], b=[], loss=[])
for sid in sids_orig:
    print(sid)
    for h in ['lh','rh']:
        for lbl in [1,2,3,4]:
            try:
                r = fit_cumarea(sid, h, lbl)
            except Exception as e:
                print(f"  - Skipping: {type(e)}")
                continue
            df['sid'].append(sid)
            df['h'].append(h)
            df['label'].append(lbl)
            df['a'].append(r.x[0])
            df['b'].append(r.x[1])
            df['loss'].append(r.fun)
HH91_params = pd.DataFrame(df)

In [ ]:
HH91_params

In [ ]:
df = HH91_params
(fig,axs) = plt.subplots(4,2, figsize=(4,4), dpi=288, sharex=True, sharey=True)
for (lbl,axrow) in zip([1,2,3,4], axs):
    df1 = df[df['label'] == lbl]
    for (h,ax) in zip(['lh','rh'], axrow):
        df2 = df1[df1['h'] == h]
        q = df2['a'].values
        ax.hist(q)
        ax.set_xlim([0,30])

In [ ]:
df = HH91_params
(fig,axs) = plt.subplots(4,2, figsize=(4,4), dpi=288, sharex=True, sharey=True)
for (lbl,axrow) in zip([1,2,3,4], axs):
    df1 = df[df['label'] == lbl]
    for (h,ax) in zip(['lh','rh'], axrow):
        df2 = df1[df1['h'] == h]
        q = df2['b'].values
        ax.hist(q)
        #ax.set_xlim([0,2])

In [ ]:
df = HH91_params
(fig,axs) = plt.subplots(4,2, figsize=(4,4), dpi=288, sharex=True, sharey=True)
fig.subplots_adjust(0,0,1,1,0.1,0.1)
eccrng = np.linspace(0.5, 11, 500)
for (lbl,axrow) in zip([1,2,3,4], axs):
    df1 = df[df['label'] == lbl]
    for (h,ax) in zip(['lh','rh'], axrow):
        df2 = df1[df1['h'] == h]
        for (a,b) in zip(df2['a'].values, df2['b'].values):
            ax.loglog(eccrng, (a / (b + eccrng))**2, 'k-', alpha=0.1)

In [ ]:
HH91_params[(HH91_params['label']==4) & (HH91_params['h']=='rh')]

In [ ]:
df = HH91_params
(fig,axs) = plt.subplots(1,2, figsize=(4,1.5), dpi=288, sharex=True, sharey=True)
eccrng = np.linspace(0.5, 12, 500)
for (lbl,clr) in zip([1,2,3,4], ['r','g','b','k']):
    df1 = df[df['label'] == lbl]
    for (h,ax) in zip(['lh','rh'], axs):
        df2 = df1[df1['h'] == h]
        a = df2['a'].values
        b = df2['b'].values
        m = (a[:,None] / (b[:,None] + eccrng[None,:]))**2
        mu = np.mean(m, axis=0)
        sd = np.std(m, axis=0)
        ax.fill_between(eccrng, mu - sd, mu + sd, edgecolor=None, facecolor=clr, alpha=0.2, zorder=-1)
        ax.loglog(eccrng, mu, clr+'-', lw=1)

## rerun analysis using the new fitting method

In [ ]:
# only look at horizontal meridian for tmp_hor
# h is assigned 'lh' for x values larger than 0 and 'rh' otherwise
tmp_hor = mean_cd_polar[mean_cd_polar['Eccen_Y']==0]
x = tmp_hor['Eccen_X'].values
# for calculating cmag, have a col for abs(x)
tmp_hor['eccen'] = np.abs(x)
tmp_hor['h'] = np.where(x > 0, 'lh', 'rh')
tmp_hor.rename(columns={'CrowdingDistance': 'CrowdingDistance_HM'}, inplace=True)
tmp_hor.rename(columns={'ID': 'sid'}, inplace=True)

In [ ]:
# only look at vertical meridian for tmp_ver
tmp_ver = mean_cd_polar[mean_cd_polar['Eccen_X']==0]
y = tmp_ver['Eccen_Y'].values
tmp_ver['eccen'] = np.abs(y)

# Create the 'CrowdingDistance_UVM' and 'CrowdingDistance_LVM' columns
tmp_ver['CrowdingDistance_UVM'] = np.where(y > 0, np.nan, tmp_ver['CrowdingDistance'])
tmp_ver['CrowdingDistance_LVM'] = np.where(y > 0, tmp_ver['CrowdingDistance'], np.nan)
tmp_ver.rename(columns={'ID': 'sid'}, inplace=True)

In [ ]:
# Create duplicate rows with 'h' column
tmp_ver_lh = tmp_ver.copy()
tmp_ver_lh['h'] = 'lh'
tmp_ver_rh = tmp_ver.copy()
tmp_ver_rh['h'] = 'rh'

# Combine the two DataFrames, so now tmp_ver has two rows with same data except one row lh and one row rh
tmp_ver = pd.concat([tmp_ver_lh, tmp_ver_rh], ignore_index=True)
tmp_ver = tmp_ver.drop(columns=['CrowdingDistance'])

In [ ]:
df = HH91_params.copy()

# will have three rows for diff ecc, but the same a,b params
df = df.merge(
    pd.DataFrame(dict(eccen=[2.5, 5.0, 10.0])),
    how='cross')

a = df['a']
b = df['b']
ecc = df['eccen']
# calculate cmag based on a,b params from HH91_params
df['cmag_fit'] = (a / (ecc + b))**2
# add 1d visual magnification
df['1d_vmag_fit'] = np.sqrt(1/ (a / (ecc + b))**2)

In [ ]:
# add CrowdingDistance_HM to df by merging it with tmp_hor
# _full versions have x, y positions too
df_HM_full = df.merge(tmp_hor, on=['sid', 'eccen','h'])

In [ ]:
df_HM = df_HM_full.drop(columns=['Eccen_X', 'Eccen_Y']) # 15 subs, 2 hemisphere * 3 ecc * 4 visual area

In [ ]:
df_VM_full = df.merge(tmp_ver, on=['sid', 'eccen','h'])

In [ ]:
df_VM = df_VM_full.drop(columns=['Eccen_X', 'Eccen_Y'])

In [ ]:
# take the average of lh and rh cmag fits
df_VM = df_VM.groupby(
    ['sid', 'label', 'eccen'], as_index=False
).agg({
    'a': 'mean',
    'b': 'mean',
    'loss': 'mean',
    'cmag_fit': 'mean',
    '1d_vmag_fit': 'mean',
    'CrowdingDistance_UVM': np.nanmean, 
    'CrowdingDistance_LVM': np.nanmean, 
})

In [ ]:
label = 4
h = 'rh'

fig, axs = plt.subplots(1,1, figsize=(4,2), dpi=288)
df_sub = df_HM[(df_HM['label']==label) & (df_HM['h']==h)]
loss = df_sub['loss']
vmag = df_sub['1d_vmag_fit']
cd = df_sub['CrowdingDistance_HM']
eccen = df_sub['eccen']

for (e,color) in zip([2.5,5,10],'rgb'):
    ii = (eccen==e)
    axs.scatter(vmag[ii], cd[ii], s=4*(2-loss[ii]/2000), c=color)
axs.plot([0,1],[0,1],'k-')
plt.show()

In [ ]:
# plots based on HM measurements
fig, axs = plt.subplots(4,2, figsize=(4,4), dpi=288, sharex=True, sharey=True)
fig.subplots_adjust(0,0,1,1,0.1,0.3)

for (lbl,axrow) in zip([1,2,3,4], axs):
    df1 = df_HM[df_HM['label'] == lbl]
    for (h,ax) in zip(['lh','rh'], axrow):
        df2 = df1[df1['h'] == h]
        loss = df2['loss']
        vmag = df2['1d_vmag_fit']
        cd = df2['CrowdingDistance_HM']
        eccen = df2['eccen']
        for (e,color) in zip([2.5,5,10],'rgb'):
            ii = (eccen==e)
            ax.scatter(vmag[ii], cd[ii], s=4*(2-loss[ii]/2000), c=color)
            ax.plot([0,2],[0,2],'k-')
            ax.set_title(f'{h} Label={lbl}', fontsize=6)
            
fig.text(0.5, -0.1, '1D Vmag Fit', ha='center', fontsize=8)
fig.text(-0.1, 0.5, 'Crowding Distance', va='center', rotation='vertical', fontsize=8)
plt.show()

In [ ]:
res = {1:[],2:[],3:[],4:[]}
for subject, subject_data in df_HM.groupby(['sid']):
    for h in ['lh','rh']:
        ssdf = subject_data[subject_data['h'] == h]
        for lbl in [1,2,3,4]:
            sssdf = ssdf[ssdf['label'] == lbl]
            x = sssdf['1d_vmag_fit'].values
            y = sssdf['CrowdingDistance_HM'].values
            (rss,coef) = cc.regression.fit_and_evaluate(x, y)
            res[lbl].append(rss)

In [ ]:
grouped = df_HM.groupby('sid') 
subject_results = {}

for subject, subject_data in grouped:
    if (subject_data['b']>5).any():
        continue
    subject_results[subject] = {}
    for h in ['lh', 'rh']:
        hemisphere_data = subject_data[subject_data['h'] == h]
        
        y1 = hemisphere_data[hemisphere_data['label'] == 1]['1d_vmag_fit'].values
        y2 = hemisphere_data[hemisphere_data['label'] == 2]['1d_vmag_fit'].values
        y3 = hemisphere_data[hemisphere_data['label'] == 3]['1d_vmag_fit'].values
        y4 = hemisphere_data[hemisphere_data['label'] == 4]['1d_vmag_fit'].values
        # since cd are the same for label 1,2,3,4
        y5 = hemisphere_data[hemisphere_data['label'] == 1]['CrowdingDistance_HM'].values
        
        rss_y1, coef_y1 = cc.regression.fit_and_evaluate(y1, y5)
        rss_y2, coef_y2 = cc.regression.fit_and_evaluate(y2, y5)
        rss_y3, coef_y3 = cc.regression.fit_and_evaluate(y3, y5)
        rss_y4, coef_y4 = cc.regression.fit_and_evaluate(y4, y5)
        
        rss_values = [rss_y1, rss_y2, rss_y3, rss_y4]
        best_fit_index = np.argmin(rss_values)
        best_fit = ['label_1', 'label_2', 'label_3', 'label_4'][best_fit_index]
        
        subject_results[subject][h] = {
            'rss_label_1': rss_y1, 'coef_label_1': coef_y1,
            'rss_label_2': rss_y2, 'coef_label_2': coef_y2,
            'rss_label_3': rss_y3, 'coef_label_3': coef_y3,
            'rss_label_4': rss_y4, 'coef_label_4': coef_y4,
            'best_fit': best_fit
        }

for subject, hemispheres in subject_results.items():
    print(f"Results for {subject}:")
    for h, results in hemispheres.items():
        print(f"  Hemisphere: {h}")
        print(f"    RSS for label 1: {results['rss_label_1']}, Coefficient: {results['coef_label_1']}")
        print(f"    RSS for label 2: {results['rss_label_2']}, Coefficient: {results['coef_label_2']}")
        print(f"    RSS for label 3: {results['rss_label_3']}, Coefficient: {results['coef_label_3']}")
        print(f"    RSS for label 4: {results['rss_label_4']}, Coefficient: {results['coef_label_4']}")
        print(f"    The best fit is: {results['best_fit']}\n")


In [ ]:
rss_keys = ['rss_label_1', 'rss_label_2', 'rss_label_3', 'rss_label_4']

data = [[
    hemispheres[h][key]  
    for subject, hemispheres in subject_results.items()
    for h in ['lh', 'rh'] 
] for key in rss_keys]

plt.figure(figsize=(10, 6))
plt.hist(data, label=['Label 1', 'Label 2', 'Label 3', 'Label 4'])
plt.xlabel('RSS Values')
plt.ylabel('Frequency')
plt.title('Distribution of RSS Values for Each Label')
plt.legend()
plt.show()

In [ ]:
rss_label_1_values = []
rss_label_2_values = []
rss_label_3_values = []
rss_label_4_values = []

for subject, hemispheres in subject_results.items():
    for h in ['lh', 'rh']:
        rss_label_1_values.append(hemispheres[h]['rss_label_1'])
        rss_label_2_values.append(hemispheres[h]['rss_label_2'])
        rss_label_3_values.append(hemispheres[h]['rss_label_3'])
        rss_label_4_values.append(hemispheres[h]['rss_label_4'])

mean_rss_label_1 = np.mean(rss_label_1_values)
mean_rss_label_2 = np.mean(rss_label_2_values)
mean_rss_label_3 = np.mean(rss_label_3_values)
mean_rss_label_4 = np.mean(rss_label_4_values)

std_rss_label_1 = np.std(rss_label_1_values)
std_rss_label_2 = np.std(rss_label_2_values)
std_rss_label_3 = np.std(rss_label_3_values)
std_rss_label_4 = np.std(rss_label_4_values)

print(f"Mean RSS for Label 1: {mean_rss_label_1:.4f}, Standard Deviation: {std_rss_label_1:.4f}")
print(f"Mean RSS for Label 2: {mean_rss_label_2:.4f}, Standard Deviation: {std_rss_label_2:.4f}")
print(f"Mean RSS for Label 3: {mean_rss_label_3:.4f}, Standard Deviation: {std_rss_label_3:.4f}")
print(f"Mean RSS for Label 4: {mean_rss_label_4:.4f}, Standard Deviation: {std_rss_label_4:.4f}")


In [ ]:
mean_rss = [mean_rss_label_1, mean_rss_label_2, mean_rss_label_3, mean_rss_label_4]
std_rss = [std_rss_label_1, std_rss_label_2, std_rss_label_3, std_rss_label_4]

labels = ['V1', 'V2', 'V3', 'V4']

# standard error of the mean (SEM)
n_subjects = len(subject_results) * 2  # for both hemispheres
sem_rss = np.array(std_rss) / np.sqrt(n_subjects)

plt.figure(figsize=(8, 6))
plt.bar(labels, mean_rss, yerr=sem_rss)

plt.ylabel('RSS (Residual Sum of Squares)')
plt.title('Mean RSS with Standard Error Across Visual Areas')

plt.show()

In [ ]:
{k:np.mean(v) for (k,v) in res.items()}

In [ ]:
{k:np.std(v) for (k,v) in res.items()}